# Buscar os dados necessários para a análise

## Gerar o dataframe com os dados da Prefeitura

In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs
import pandas as pd
from io import StringIO
import chardet

### Criar o dataframe que conterá os dados necessários para a análise

In [2]:
df = pd.DataFrame()
dfs = []

### Função para buscar os caminhos para o download de cada dataset

In [3]:
def get_soup(url, search):
    response = requests.get(url)
    
    # Verificar se a conexão foi bem sucedida
    if response.status_code == 200:
        soup = bs(response.content, 'html.parser')

        # Retornar as páginas onde terão as de download
        return soup.find_all('a', href=True, attrs={'title': re.compile(search)}) 
    else:
        return f'Página{link["title"]} está inacessível. Código: {response.status_code}'

### Código para inserir os dados em um dataframe

In [4]:
# URL base do site da Prefeitura
url_base = 'http://dados.prefeitura.sp.gov.br'

# URL inicial onde estarão os links para os datasets
url_inicial = 'http://dados.prefeitura.sp.gov.br/dataset/dados-do-sp156'

# extensão do dataset para filtrar
file_type = '.csv'

for link in get_soup(url_inicial, r'^Dados do SP156 -.*'):
    
    # Pegar o link de download
    for page in get_soup(url_base+link['href'], r'^http://dados.prefeitura.sp.gov.br/dataset/.*'):
        link_download = page['href']
        
        # Verificar se é o arquivo a ser baixado é Excel
        if file_type in link_download:
            
            # Pegar o conteúdo do CSV
            response_csv = requests.get(page['href'])
            
            # Cada datset tem um encoding diferente, por isso foi necessário usar a biblioteca chardet para identificar 
            # automaticamente qual é.
            encoding = chardet.detect(response_csv.content)['encoding']
            
            # Colocar os dados na tabela auxiliar
            if int(link["title"][-4:]) >= 2021:
                df3 = pd.read_csv(StringIO(response_csv.content.decode(encoding)), sep=';', encoding=encoding, low_memory=False)
            else:
                df3 = pd.read_csv(StringIO(response_csv.content.decode(encoding)), sep=',', encoding=encoding, low_memory=False)
                
            dfs.append(df3)  # Adicionar o DataFrame intermediário à lista
            
            # Inserir os dados na planilha original
            print(f'Os dados do {link["title"]} foram inseridos com sucesso.')

Os dados do Dados do SP156 - 1º TRI 2023 foram inseridos com sucesso.
Os dados do Dados do SP156 - 4º TRI 2022 foram inseridos com sucesso.
Os dados do Dados do SP156 - 3º TRI 2022 foram inseridos com sucesso.
Os dados do Dados do SP156 - 2º TRI 2022 foram inseridos com sucesso.
Os dados do Dados do SP156 - 1º TRI 2022 foram inseridos com sucesso.
Os dados do Dados do SP156 - 4º TRI 2021 foram inseridos com sucesso.
Os dados do Dados do SP156 - 3º TRI 2021 foram inseridos com sucesso.
Os dados do Dados do SP156 - 2º TRI 2021 foram inseridos com sucesso.
Os dados do Dados do SP156 - 1º TRI 2021 foram inseridos com sucesso.
Os dados do Dados do SP156 - Dezembro - 2020 foram inseridos com sucesso.
Os dados do Dados do SP156 - Outubro e Novembro - 2020 foram inseridos com sucesso.
Os dados do Dados do SP156 - 3º TRI 2020 foram inseridos com sucesso.
Os dados do Dados do SP156 - 1º SEM 2020 foram inseridos com sucesso.
Os dados do Dados do SP156 - 2º SEM 2019 foram inseridos com sucesso.
Os

## Criar o dataframe com todos os datasets coletados

In [5]:
df = pd.concat(dfs, ignore_index=True)

In [6]:
df.shape

(13688540, 33)

## Verificar todas as colunas disponibilizadas

In [7]:
df.any()

Data de abertura                                                          True
Canal                                                                     True
Tema                                                                      True
Assunto                                                                   True
Serviço                                                                   True
Logradouro                                                                True
Número                                                                    True
CEP                                                                       True
Subprefeitura                                                             True
Distrito                                                                  True
Setor                                                                     True
Quadra                                                                    True
Latitude                                            

## Manter apenas os campos necessários

In [8]:
df = df[['Data de abertura', 'Tema', 'Assunto', 'Especificação do assunto', 'Serviço', 'Bairro', 'Distrito'
         , 'Status da solicitação', 'Data do parecer']]

## Apagar registros que por terem muitos valores nulo, não servirão para a análise

In [9]:
df = df.dropna(thresh=4)

In [41]:
df[df.Assunto.isnull()]

,Data de abertura,Tema,Assunto,Especificação do assunto,Serviço,Bairro,Distrito,Status da solicitação,Data do parecer


In [10]:
df.sample(10)

,Data de abertura,Tema,Assunto,Especificação do assunto,Serviço,Bairro,Distrito,Status da solicitação,Data do parecer
5491471,2020-12-10 02:02:27.005,Cidadania e assistência social,Renda Básica Emergencial,NaN,Renda Básica Emergencial - Consultar benefício...,NaN,NaN,FINALIZADA,2020-12-10 02:02:29.657
7367506,2020-05-12 11:03:07.594,Trânsito e Transporte,Sinalização e Circulação de Veículos e Pedestres,NaN,Rodizio de Veiculos (coronavírus) - Cadastrar ...,NaN,NaN,FINALIZADA,2020-05-16 20:24:23.386
8314883,2019-03-06 12:19:30.454,Trânsito e Transporte,"Ônibus - Conduta de Trabalho do Motorista, Cob...",NaN,Ônibus - Denunciar motorista por direção perigosa,NaN,CAMPO LIMPO,FINALIZADA,2019-03-21 21:10:43.203
4917495,2021-03-02 11:25:02,Cidadania e assistência social,Renda Básica Emergencial,NaN,Renda Básica Emergencial - Consultar benefício...,NaN,NaN,FINALIZADA,2021-03-02 11:25:05
11298000,2015-05-09 19:53:14.100,Bilhete Único,BU Cobrança Carga e Recarga,NaN,Comunicação de erro em cobrança de integração ...,NaN,SANTO AMARO,FINALIZADA,2015-05-29 14:25:00.000
4931375,2021-03-03 13:24:07,Cidadania e assistência social,Renda Básica Emergencial,NaN,Renda Básica Emergencial - Consultar benefício...,NaN,NaN,FINALIZADA,2021-03-03 13:24:10
1585985,2022-04-19 17:28:11,Rua e bairro,Capinação e roçada de áreas verdes,NaN,Guias e sarjetas - Solicitar capinação,NaN,NaN,FINALIZADA,2022-04-20 11:39:50
741805,2022-10-31 11:43:16,Finanças,Senha Web,NaN,"Senha Web - Desbloqueio online para MEI, Empre...",NaN,NaN,INDEFERIDO,2022-11-02 01:35:12
4611782,2021-01-26 16:34:56,Cidadania e assistência social,Renda Básica Emergencial,NaN,Renda Básica Emergencial - Consultar benefício...,NaN,NaN,FINALIZADA,2021-01-26 16:34:57
9774163,2017-12-22 23:29:00.096,Rua e bairro,Poluição Sonora - PSIU,NaN,Reclamação de poluição sonora - PSIU,NaN,CASA VERDE,FINALIZADA,2020-03-26 23:34:15.889


## Filtrar apenas os dados necessários para a análise

In [11]:
df['Tema'].drop_duplicates()

0           Cidadania e assistência social
5                                  Animais
7               Obras, imóveis e habitação
10                            Rua e bairro
34                           Meio Ambiente
37                          Lixo e limpeza
41                                Urgência
54                                   Saúde
67                     Comércio e serviços
95                   Trânsito e Transporte
151                           Causa animal
188           Ouvidoria Geral do Município
227                         Pragas urbanas
431                               Finanças
439                         Acessibilidade
918                          Bilhete Único
1272                      Segurança urbana
2334                 Documentos e licenças
2885              Cultura, esporte e lazer
4389                              Educação
8048                              Trabalho
334059              Descomplica São Miguel
596920               Canais de Atendimento
597569     

### Entender o motivo de existirem nulos no 'Tema'

In [52]:
pd.set_option('display.max_rows', None)
df[df.Tema.isnull()]['Assunto'].drop_duplicates()

11388300                             Construcao de passarelas
11388301                                               Buraco
11388302                                      Poluição Sonora
11388304    Placas de Nomes de Ruas, Praças e demais Logra...
11388306                         Iluminacao - Lampada apagada
11388307             Transporte publico / Conduta de Trabalho
11388308                              Remoção veículo/carcaça
11388310                                               Elogio
11388314                            Vistoria / Infraestrutura
11388317                                                Obras
11388318                                           Edificacao
11388319                                        Animais / Cão
11388324                                         Pavimentacao
11388329                      Árvore(Calçada/Passeio Público)
11388332                          Transporte publico / Outros
11388334                                       Guias/sarjetas
11388336

In [13]:
df1 = df[df['Assunto'].str.contains('Animais', regex=False) | df['Tema'].str.contains(r'^Anim.*|^Pragas.*', regex=True)].reset_index(drop=True)

In [16]:
df1.shape

(704360, 9)

In [8]:
df.sample(20)

,Data de abertura,Tema,Assunto,Especificação do assunto,Serviço,Bairro,Distrito,Status da solicitação,Data do parecer
52299,2020-09-17 20:39:00.971,Animais,"Exames, vacinas e castração",NaN,Castrar cães e gatos gratuitamente,NaN,NaN,INDEFERIDO,2020-11-30 13:34:28.525
180895,2017-02-20 19:23:56.245,Animais,Animais que transmitem doenças ou risco à saúde,NaN,Abelhas e Vespas - Solicitar vistoria em ninhos,NaN,JARDIM HELENA,FINALIZADA,2017-02-24 17:23:00.446
193620,2016-01-11 10:36:56.637,Animais,Animal em via pública,NaN,Avaliação de animal em sofrimento sem propriet...,NaN,CAMPO LIMPO,FINALIZADA,2017-06-19 13:07:38.990
149729,2018-04-07 11:20:45.877,Animais,Criação inadequada de animais,NaN,Denunciar condições inadequadas de criação,NaN,ITAQUERA,FINALIZADA,2018-05-02 11:43:25.307
436902,2012-05-19 11:11:11.023,NaN,Animais / Pombos,Reclamação de Pombos,NaN,CONJUNTO RESIDENCIAL VISTA VERDE,64.0,REALIZADA,2012-06-04 00:00:00.000
118015,2019-06-05 15:55:02.335,Animais,Animal em via pública,NaN,Avaliação de animal em sofrimento sem propriet...,NaN,SAUDE,FINALIZADA,2020-02-14 18:19:17.607
383407,2012-07-25 09:59:07.083,NaN,Animais / Cão,Atropelado ou Acidentado vivo e sem proprietário,NaN,LAUZANE PAULISTA,51.0,CANCELADA,2012-07-25 00:00:00.000
199373,2016-02-13 15:13:44.133,Animais,Animais que transmitem doenças ou risco à saúde,NaN,Abelhas e Vespas - Solicitar vistoria em ninhos,NaN,ARICANDUVA,FINALIZADA,2016-04-11 13:36:03.013
101548,2019-01-18 18:30:39.845,Animais,Animal em via pública,NaN,Solicitar remoção de animal morto em via pública,NaN,ITAIM BIBI,FINALIZADA,2019-02-06 15:59:39.000
82171,2019-08-09 16:15:04.514,Animais,Animais que transmitem doenças ou risco à saúde,NaN,Abelhas e Vespas - Solicitar vistoria em ninhos,NaN,MOEMA,FINALIZADA,2019-08-13 13:25:07.165


In [43]:
df[df['Especificação do assunto'].isnull()]

,Data de abertura,Tema,Assunto,Especificação do assunto,Serviço,Bairro,Distrito,Status da solicitação,Data do parecer
0,2022-12-31 21:07:05,Cidadania e assistência social,População ou pessoa em situação de rua,NaN,Pessoa em situação de rua ? Solicitar atendime...,NaN,NaN,FINALIZADA,2022-12-31 21:52:01
1,2022-12-31 21:13:26,Cidadania e assistência social,População ou pessoa em situação de rua,NaN,Pessoa em situação de rua ? Solicitar atendime...,NaN,NaN,FINALIZADA,2022-12-31 23:08:36
2,2022-12-31 21:15:16,Cidadania e assistência social,População ou pessoa em situação de rua,NaN,Pessoa em situação de rua ? Solicitar atendime...,NaN,BELEM,FINALIZADA,2023-01-01 01:06:00
3,2022-12-31 21:16:08,Cidadania e assistência social,População ou pessoa em situação de rua,NaN,Pessoa em situação de rua ? Solicitar atendime...,NaN,SE,FINALIZADA,2022-12-31 23:09:19
4,2022-12-31 21:22:07,Cidadania e assistência social,População ou pessoa em situação de rua,NaN,Pessoa em situação de rua ? Solicitar atendime...,NaN,NaN,FINALIZADA,2022-12-31 22:04:24
...,...,...,...,...,...,...,...,...,...
11388295,2015-07-01 02:56:47.833,Rua e bairro,Poluição Sonora - PSIU,NaN,Reclamação de poluição sonora - PSIU,NaN,PEDREIRA,FINALIZADA,2015-07-28 13:59:43.360
11388296,2015-07-01 02:56:56.000,Trânsito e Transporte,Veículos abandonados,NaN,Remoção de veículo/carcaça abandonado na via p...,NaN,CIDADE LIDER,FINALIZADA,2018-07-16 21:13:09.000
11388297,2015-07-01 02:57:10.710,Lixo e limpeza,Varrição e limpeza urbana,NaN,Solicitar limpeza da via pública após enchente...,NaN,PENHA,FINALIZADA,2015-07-13 03:00:00.000
11388298,2015-07-01 02:59:26.000,Trânsito e Transporte,Veículos abandonados,NaN,Remoção de veículo/carcaça abandonado na via p...,NaN,TREMEMBE,FINALIZADA,2015-09-11 15:53:43.460


## Gerar o arquivo com os dados necessários

In [10]:
df.to_csv('sp156_all_time.csv', sep=';')